In [5]:
# Import python modules we need numpy, pandas, and sklearn

In [6]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

In [7]:
# Read in data files. Columns in the file: mu1=m2/(m1+m2), mu2=m3/(m1+m2), I, e1, e2, node, pericentre of planet, inner border, outer border

In [8]:
df=pd.read_csv('bordptncee.out', sep='\s+', names=['mu1','mu2','inc_deg','e1','e2','node_deg','w2_deg','border_i','border_o'])

In [9]:
df

,mu1,mu2,inc_deg,e1,e2,node_deg,w2_deg,border_i,border_o
0,0.50,1.000000e-02,18.0,0.1,0.1,0.0,0.0,2.7,3.4
1,0.50,1.000000e-02,18.0,0.1,0.1,0.0,90.0,2.8,3.4
2,0.50,1.000000e-02,18.0,0.1,0.1,0.0,180.0,2.7,3.4
3,0.50,1.000000e-02,18.0,0.1,0.1,90.0,0.0,2.6,3.2
4,0.50,1.000000e-02,18.0,0.1,0.1,90.0,90.0,2.7,3.4
...,...,...,...,...,...,...,...,...,...
236191,0.01,1.000000e-07,162.0,0.9,0.9,90.0,90.0,29.3,38.8
236192,0.01,1.000000e-07,162.0,0.9,0.9,90.0,180.0,39.3,46.3
236193,0.01,1.000000e-07,162.0,0.9,0.9,180.0,0.0,39.1,52.8
236194,0.01,1.000000e-07,162.0,0.9,0.9,180.0,90.0,34.9,43.8


In [10]:
# this is how you would export pandas dataframe to csv (uncomment next line)
# df.to_csv('georgakarakos_data.csv',index=False)

In [11]:
# Create multi index to be able to easily reorder the file for averaging
dfmi=pd.MultiIndex.from_frame(df)

In [12]:
dfmi

MultiIndex([( 0.5,  0.01,  18.0, 0.1, 0.1,   0.0,   0.0,  2.7,  3.4),
            ( 0.5,  0.01,  18.0, 0.1, 0.1,   0.0,  90.0,  2.8,  3.4),
            ( 0.5,  0.01,  18.0, 0.1, 0.1,   0.0, 180.0,  2.7,  3.4),
            ( 0.5,  0.01,  18.0, 0.1, 0.1,  90.0,   0.0,  2.6,  3.2),
            ( 0.5,  0.01,  18.0, 0.1, 0.1,  90.0,  90.0,  2.7,  3.4),
            ( 0.5,  0.01,  18.0, 0.1, 0.1,  90.0, 180.0,  2.6,  3.2),
            ( 0.5,  0.01,  18.0, 0.1, 0.1, 180.0,   0.0,  2.7,  3.4),
            ( 0.5,  0.01,  18.0, 0.1, 0.1, 180.0,  90.0,  2.8,  3.4),
            ( 0.5,  0.01,  18.0, 0.1, 0.1, 180.0, 180.0,  2.7,  3.4),
            ( 0.5,  0.01,  18.0, 0.1, 0.2,   0.0,   0.0,  3.3,  3.8),
            ...
            (0.01, 1e-07, 162.0, 0.8, 0.9, 180.0, 180.0, 25.0, 33.1),
            (0.01, 1e-07, 162.0, 0.9, 0.9,   0.0,   0.0, 29.0, 34.6),
            (0.01, 1e-07, 162.0, 0.9, 0.9,   0.0,  90.0, 36.1, 41.9),
            (0.01, 1e-07, 162.0, 0.9, 0.9,   0.0, 180.0, 43.8, 51.9),
    

In [13]:
help(dfmi)

Help on MultiIndex in module pandas.core.indexes.multi object:

class MultiIndex(pandas.core.indexes.base.Index)
 |  A multi-level, or hierarchical, index object for pandas objects.
 |  
 |  Parameters
 |  ----------
 |  levels : sequence of arrays
 |      The unique labels for each level.
 |  codes : sequence of arrays
 |      Integers for each level designating which label at each location.
 |  
 |      .. versionadded:: 0.24.0
 |  sortorder : optional int
 |      Level of sortedness (must be lexicographically sorted by that
 |      level).
 |  names : optional sequence of objects
 |      Names for each of the index levels. (name is accepted for compat).
 |  copy : bool, default False
 |      Copy the meta-data.
 |  verify_integrity : bool, default True
 |      Check that the levels/codes are consistent and valid.
 |  
 |  Attributes
 |  ----------
 |  names
 |  levels
 |  codes
 |  nlevels
 |  levshape
 |  
 |  Methods
 |  -------
 |  from_arrays
 |  from_tuples
 |  from_product
 | 

In [14]:
dfmi2=dfmi.reorder_levels(['mu1', 'mu2', 'inc_deg', 'e1', 'node_deg','w2_deg','e2','border_i', 'border_o'])

In [15]:
dfmi2sorted=dfmi2.sort_values([('mu1', 'mu2', 'inc_deg', 'e1', 'node_deg','w2_deg')], ascending=True)

In [16]:
len(dfmi2sorted[1])

236196

In [17]:
dfmi2sorted[0][0:9]

MultiIndex([(0.01, 1e-07, 18.0, 0.1, 0.0, 0.0, 0.1,  1.7,  1.8),
            (0.01, 1e-07, 18.0, 0.1, 0.0, 0.0, 0.2,  1.9,  2.6),
            (0.01, 1e-07, 18.0, 0.1, 0.0, 0.0, 0.3,  2.5,  3.0),
            (0.01, 1e-07, 18.0, 0.1, 0.0, 0.0, 0.4,  3.1,  4.1),
            (0.01, 1e-07, 18.0, 0.1, 0.0, 0.0, 0.5,  4.0,  5.0),
            (0.01, 1e-07, 18.0, 0.1, 0.0, 0.0, 0.6,  5.9,  6.7),
            (0.01, 1e-07, 18.0, 0.1, 0.0, 0.0, 0.7,  8.8,  9.7),
            (0.01, 1e-07, 18.0, 0.1, 0.0, 0.0, 0.8, 13.6, 16.7),
            (0.01, 1e-07, 18.0, 0.1, 0.0, 0.0, 0.9, 32.3, 38.8)],
           names=['mu1', 'mu2', 'inc_deg', 'e1', 'node_deg', 'w2_deg', 'e2', 'border_i', 'border_o'])

In [18]:
dfmi2sorted

(MultiIndex([(0.01, 1e-07,  18.0, 0.1,   0.0,   0.0, 0.1,  1.7,  1.8),
             (0.01, 1e-07,  18.0, 0.1,   0.0,   0.0, 0.2,  1.9,  2.6),
             (0.01, 1e-07,  18.0, 0.1,   0.0,   0.0, 0.3,  2.5,  3.0),
             (0.01, 1e-07,  18.0, 0.1,   0.0,   0.0, 0.4,  3.1,  4.1),
             (0.01, 1e-07,  18.0, 0.1,   0.0,   0.0, 0.5,  4.0,  5.0),
             (0.01, 1e-07,  18.0, 0.1,   0.0,   0.0, 0.6,  5.9,  6.7),
             (0.01, 1e-07,  18.0, 0.1,   0.0,   0.0, 0.7,  8.8,  9.7),
             (0.01, 1e-07,  18.0, 0.1,   0.0,   0.0, 0.8, 13.6, 16.7),
             (0.01, 1e-07,  18.0, 0.1,   0.0,   0.0, 0.9, 32.3, 38.8),
             (0.01, 1e-07,  18.0, 0.1,   0.0,  90.0, 0.1,  1.9,  2.6),
             ...
             ( 0.5,  0.01, 162.0, 0.9, 180.0,  90.0, 0.9, 48.8, 55.0),
             ( 0.5,  0.01, 162.0, 0.9, 180.0, 180.0, 0.1,  3.6,  4.5),
             ( 0.5,  0.01, 162.0, 0.9, 180.0, 180.0, 0.2,  4.2,  5.0),
             ( 0.5,  0.01, 162.0, 0.9, 180.0, 180.0, 0.3,  4

In [19]:
df2=dfmi2sorted[0].to_frame(index=False)

In [20]:
# if you want to average over e2 uncomment the next tow lines
# df3=df2.groupby(['mu1', 'mu2', 'inc_deg', 'e1', 'node_deg','w2_deg']).mean()
# df4=df3.add_suffix('_mean').reset_index()

In [21]:
df4=df2

In [22]:
df4

,mu1,mu2,inc_deg,e1,node_deg,w2_deg,e2,border_i,border_o
0,0.01,1.000000e-07,18.0,0.1,0.0,0.0,0.1,1.7,1.8
1,0.01,1.000000e-07,18.0,0.1,0.0,0.0,0.2,1.9,2.6
2,0.01,1.000000e-07,18.0,0.1,0.0,0.0,0.3,2.5,3.0
3,0.01,1.000000e-07,18.0,0.1,0.0,0.0,0.4,3.1,4.1
4,0.01,1.000000e-07,18.0,0.1,0.0,0.0,0.5,4.0,5.0
...,...,...,...,...,...,...,...,...,...
236191,0.50,1.000000e-02,162.0,0.9,180.0,180.0,0.5,7.4,8.1
236192,0.50,1.000000e-02,162.0,0.9,180.0,180.0,0.6,9.7,10.9
236193,0.50,1.000000e-02,162.0,0.9,180.0,180.0,0.7,13.9,15.6
236194,0.50,1.000000e-02,162.0,0.9,180.0,180.0,0.8,23.1,25.5


In [ ]:
# Fitting starts here

In [24]:
X = df4[['mu1', 'mu2', 'inc_deg', 'e1', 'node_deg','w2_deg','e2']]
X.shape

(236196, 7)

In [26]:
y = df4['border_i']
y.shape

(236196,)

In [80]:
# define highest degree of polynomial
degr=2

model = make_pipeline(StandardScaler(),PolynomialFeatures(degr), Ridge(alpha=1e-3))
#model = make_pipeline(StandardScaler(),PolynomialFeatures(degr), linear_model.LinearRegression())
model.fit(X, y)
y_c = model.predict(X)


In [81]:
# those are predicted values from our model
y_c

array([ 3.66118264,  0.22572121, -1.21130754, ..., 17.2273462 ,
       26.58273141, 37.93654929])

In [82]:
# See how well our model is doing against real data
model.score(X,y)

0.8847822950239376

In [83]:
# These are the polynomal coefficients of the fit
print(model['polynomialfeatures'].powers_)

[[0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1]
 [2 0 0 0 0 0 0]
 [1 1 0 0 0 0 0]
 [1 0 1 0 0 0 0]
 [1 0 0 1 0 0 0]
 [1 0 0 0 1 0 0]
 [1 0 0 0 0 1 0]
 [1 0 0 0 0 0 1]
 [0 2 0 0 0 0 0]
 [0 1 1 0 0 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 1 0 0]
 [0 1 0 0 0 1 0]
 [0 1 0 0 0 0 1]
 [0 0 2 0 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 0 1 0 0 1 0]
 [0 0 1 0 0 0 1]
 [0 0 0 2 0 0 0]
 [0 0 0 1 1 0 0]
 [0 0 0 1 0 1 0]
 [0 0 0 1 0 0 1]
 [0 0 0 0 2 0 0]
 [0 0 0 0 1 1 0]
 [0 0 0 0 1 0 1]
 [0 0 0 0 0 2 0]
 [0 0 0 0 0 1 1]
 [0 0 0 0 0 0 2]]


In [84]:
regress_coefs = model['ridge'].coef_
regress_intercept = model['ridge'].intercept_ 

In [85]:
def reduced_chi2(O,C,nparam):
    res=O-C
    chi2=np.dot(res,res)/(len(res)-nparam)
    return chi2



In [86]:
# calculate reduced chi square metric to see if we fit well (should be around 1. chi2 > 1 means underfitting - not all features are captured, chi2 < than 1 overfitting)
reduced_chi2(y,y_c,len(regress_coefs))

17.40178919019221